## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent Basic Example

This notebook demonstrates basic usage of AzureAIAgentClient to create agents with automatic lifecycle management. It shows both streaming and non-streaming responses with function tools.

## Features Covered:
- Creating an Azure AI Agent with automatic lifecycle management
- Using function tools (weather function)
- Non-streaming responses (get complete result at once)
- Streaming responses (get results as they are generated)
- Authentication using Azure CLI credentials

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the agent-framework packages
2. Authenticated with Azure CLI (`az login --use-device-code`)
3. Configured your Azure AI services

## Import Required Libraries

First, let's import all the necessary libraries and modules:

In [1]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
from random import randint
from typing import Annotated

from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Initial Setup
We'll start by importing needed libraries, loading environment variables, and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉

In [2]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import InteractiveBrowserCredential  # For Azure authentication

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

# # Get tenant ID for authentication
# tenant_id = os.environ.get("TENANT_ID")
# conn_string = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

# print(f"🔑 Using Tenant ID: {tenant_id}")

# # Initialize the AI Project Client using simplified browser-based authentication
# try:
#     print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
#     # Use only InteractiveBrowserCredential with the specific tenant
#     credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
#     # Create the project client using endpoint (conn_string is actually the endpoint URL)
#     project_client = AIProjectClient(
#         endpoint=conn_string,
#         credential=credential
#     )
#     print("✅ Successfully initialized AIProjectClient")
# except Exception as e:
#     # Print error message if client initialization fails
#     print(f"❌ Error initializing project client: {str(e)}")
#     print("💡 Please complete the browser authentication prompt that should appear")

False

## Define Function Tools

Function tools allow the agent to call specific functions to gather information or perform actions. Here we define a simple weather function that the agent can use:

In [3]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Non-Streaming Response Example

In this example, we'll create an agent and get a complete response at once (non-streaming). The agent will be automatically created and deleted after getting the response:

In [4]:
async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    # Since no Agent ID is provided, the agent will be automatically created
    # and deleted after getting a response
    # For authentication, run `az login` command in terminal or replace AzureCliCredential with preferred
    # authentication option.
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Seattle?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}\n")

## Streaming Response Example

In this example, we'll demonstrate streaming responses where we get results as they are generated by the agent:

In [5]:
async def streaming_example() -> None:
    """Example of streaming response (get results as they are generated)."""
    print("=== Streaming Response Example ===")

    # Since no Agent ID is provided, the agent will be automatically created
    # and deleted after getting a response
    # For authentication, run `az login` command in terminal or replace AzureCliCredential with preferred
    # authentication option.
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Portland?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

## Main Execution Function

This function orchestrates the execution of both examples:

In [6]:
async def main() -> None:
    print("=== Basic Azure AI Chat Client Agent Example ===")

    await non_streaming_example()
    await streaming_example()

## Run the Examples

Execute the main function to run both streaming and non-streaming examples:

In [7]:
# Run the main function
await main()

=== Basic Azure AI Chat Client Agent Example ===
=== Non-streaming Response Example ===
User: What's the weather like in Seattle?


[2026-02-12 11:25:02 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Agent: The weather in Seattle is stormy today with a high temperature of 23°C. Stay safe and prepared!

=== Streaming Response Example ===
User: What's the weather like in Portland?
Agent: 

[2026-02-12 11:25:18 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:704 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


The weather in Portland is rainy, with a high of 19°C. Stay dry!



## Key Takeaways

1. **Automatic Lifecycle Management**: When no Agent ID is provided, the agent is automatically created and cleaned up
2. **Function Tools**: Agents can use custom functions to perform specific tasks (like getting weather data)
3. **Authentication**: Uses Azure CLI credentials for authentication (make sure to run `az login` first)
4. **Response Types**: 
   - Non-streaming: Get complete response at once using `agent.run()`
   - Streaming: Get response chunks as they're generated using `agent.run_stream()`
5. **Context Managers**: Using `async with` ensures proper resource cleanup